In [1]:
# If the below code doesn't work uncomment this and run (this adds the parent directory to the path)
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import matplotlib.pyplot as plt
from BasicSetupUtilities.MetaDataBuilder import CountryMetaDataFile
from DataIOUtilities.DataLib import DataLib, DatastreamPulls
import pandas as pd
import numpy as np

dl = DataLib("SignalData")

In [2]:
'''
takes in indicator. Then processes return and plots bond return, indicator, and pnl
indicator is a function, returns new time series with the indicator

CURRENTLY ONLY DOING FOR USA
'''
bond_return_index = dl.pull('BondRetIdx/LocalFX')
bond_return_change = bond_return_index.resample('M').mean().pct_change()


def evaluate_performance(indicator):
    shifted_bond_return = bond_return_change.shift(1)
    days_profit = shifted_bond_return['USA'] * indicator
    pnl = days_profit.cumsum()
#     pnl = pnl * 100
    return pnl
    
    
def plot_evaluation(indicator, pnl, plot_indicator = False):
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.plot(pnl.to_timestamp(), label = 'Portfolio Returns')
    ax1.plot(bond_return_index['USA'].to_timestamp(), label = 'Bond Market Index')
    ax1.set_ylabel('Returns or Index')
    ax1.legend()
    if plot_indicator:
        ax2.plot(indicator.to_timestamp(), color = 'tab:red',label = 'Indicator')
        ax2.set_ylabel('Indicator')
        ax2.legend()
    plt.title('USA portfolio performance vs bond market')
    plt.show()
    std_portfolio = indicator.rolling(12).std()
    std_market = bond_return_index['USA'].resample('M').mean().rolling(12).std()
#     print(std_portfolio.tail())
#     print(std_market.tail())
    vol_adjusted_pnl = pnl * std_market / std_portfolio
    vol_adjusted_pnl.plot(label = 'vol adjusted portfolio returns')
    bond_return_index['USA'].plot(label = 'bond market returns')
    plt.legend()
    plt.title('USA bond market returns vs vol adjusted portfolio returns')
    plt.show()

In [3]:
curve_height_indicator = pd.Series([])
curr_acct_pct = dl.pull('CurrAcctPctGDP') #quarterly
curr_acct_pct_4m = curr_acct_pct
curr_acct_pct_1yr = curr_acct_pct.rolling(4).mean()
print(curr_acct_pct.head())


country    AUS  DEU    CAN  ESP  FRA  ITA  JPN  CHE    GBR    USA
date                                                             
1980Q1  -0.963  NaN    NaN  NaN  NaN  NaN  NaN  NaN -0.572 -0.496
1980Q2  -1.720  NaN    NaN  NaN  NaN  NaN  NaN  NaN -0.814 -0.135
1980Q3  -3.787  NaN    NaN  NaN  NaN  NaN  NaN  NaN  1.163  0.607
1980Q4  -3.218  NaN    NaN  NaN  NaN  NaN  NaN  NaN  2.205  0.319
1981Q1  -2.825  NaN -4.141  NaN  NaN  NaN  NaN  NaN  3.822  0.123
